In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import math
import copy
import sklearn
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import torchtext
from torchtext.data import get_tokenizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [ ]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [ ]:
#The emotions are classified into six categories: sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5).
data = pd.read_csv("/content/dataset.csv")

In [ ]:
data.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  416809 non-null  int64 
 1   text        416809 non-null  object
 2   label       416809 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 9.5+ MB


In [ ]:
data.isna().sum()

Unnamed: 0    0
text          0
label         0
dtype: int64

In [ ]:
#The emotions are classified into six categories: sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5).
data['label'].value_counts()

label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64

In [ ]:
stop_words = stopwords.words('english')
def clean_data(tweet) -> str:
  #Convert to Lower Case
  tweet = tweet.lower()

  #Remove Punctuation from the tweet
  tweet = tweet.translate(str.maketrans('', '', string.punctuation))

  #Remove stopwords from the tweet
  remove_stopwords = [word for word in tweet.split() if word not in stop_words]
  tweet = ' '.join(remove_stopwords)

  #Remove digits
  tweet = re.sub(r"\d",'',tweet)

  #remove extra space
  tweet = re.sub(r"\s+"," ",tweet)

  return tweet

In [ ]:
data['model_input_text'] = data['text'].apply(clean_data)
data.head()

,Unnamed: 0,text,label,model_input_text
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught
3,3,i dont know i feel so lost,0,dont know feel lost
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...


In [ ]:
from sentence_transformers import SentenceTransformer
# Load a pre-trained model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def embedding(sentence):
  embeddings = model.encode(sentence)
  return embeddings
def to_tensor(arr):
  return torch.tensor(arr)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
data['embedded'] = data['model_input_text'].apply(embedding)
data.head()

,Unnamed: 0,text,label,model_input_text,embedded
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[0.19374932, 0.33269206, 0.07816112, -0.028441..."
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[0.0833772, -0.27273348, 0.64947385, 0.0682905..."
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[-0.2025333, 0.15025133, 0.65832657, 0.0836372..."
3,3,i dont know i feel so lost,0,dont know feel lost,"[0.4032204, -0.06682834, 0.3310123, 0.20955956..."
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[0.1375044, 0.27389878, 0.21386795, 0.41600043..."


In [ ]:
data['embedding'] = data['embedded'].apply(to_tensor)
data.head()

,Unnamed: 0,text,label,model_input_text,embedded,embedding
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[0.19374932, 0.33269206, 0.07816112, -0.028441...","[tensor(0.1937), tensor(0.3327), tensor(0.0782..."
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[0.0833772, -0.27273348, 0.64947385, 0.0682905...","[tensor(0.0834), tensor(-0.2727), tensor(0.649..."
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[-0.2025333, 0.15025133, 0.65832657, 0.0836372...","[tensor(-0.2025), tensor(0.1503), tensor(0.658..."
3,3,i dont know i feel so lost,0,dont know feel lost,"[0.4032204, -0.06682834, 0.3310123, 0.20955956...","[tensor(0.4032), tensor(-0.0668), tensor(0.331..."
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[0.1375044, 0.27389878, 0.21386795, 0.41600043...","[tensor(0.1375), tensor(0.2739), tensor(0.2139..."


In [ ]:
#In order to define Position Encoding, we can call function get_positional_encoding function below with seq_len = Number of Sentences in each input. However in this task of classifying tweets
#We know that each input has exactly one sentence only. So positional encoding is redundant for this example since we are using sentence level embedding. So positional encoding step is skipped
#for this task. If we did a Word level embedding in this task, then we would have needed to call positional encoding in our example and then seq_len = Number of words. But that is not the case since
#we used Sentence transformer model which finds the embedding for sentences.

# def get_positional_encodings(seq_len=1, embedding_dim=384):
#     pos_enc = np.zeros((seq_len, embedding_dim))
#     for pos in range(seq_len):
#         for i in range(0, embedding_dim, 2):
#             pos_enc[pos, i] = np.sin(pos / (10000 ** ((2 * i)/embedding_dim)))
#             if i + 1 < embedding_dim:
#                 pos_enc[pos, i + 1] = np.cos(pos / (10000 ** ((2 * (i + 1))/embedding_dim)))
#     return pos_enc

# position_encoding = get_positional_encodings()

# def combined_embedding(embedding):
#   comb_embedding = embedding + position_encoding.squeeze(0)
#   return comb_embedding


# data['combined_embedded'] = data['embedded'].apply(combined_embedding)
# data.head()

In [ ]:
data.head()

,Unnamed: 0,text,label,model_input_text,embedded,embedding
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[0.19374932, 0.33269206, 0.07816112, -0.028441...","[tensor(0.1937), tensor(0.3327), tensor(0.0782..."
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[0.0833772, -0.27273348, 0.64947385, 0.0682905...","[tensor(0.0834), tensor(-0.2727), tensor(0.649..."
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[-0.2025333, 0.15025133, 0.65832657, 0.0836372...","[tensor(-0.2025), tensor(0.1503), tensor(0.658..."
3,3,i dont know i feel so lost,0,dont know feel lost,"[0.4032204, -0.06682834, 0.3310123, 0.20955956...","[tensor(0.4032), tensor(-0.0668), tensor(0.331..."
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[0.1375044, 0.27389878, 0.21386795, 0.41600043...","[tensor(0.1375), tensor(0.2739), tensor(0.2139..."


In [30]:
class MultiHead_Attention(nn.Module):
  def __init__(self,input_dimension,num_heads):
    super(MultiHead_Attention,self).__init__()
    assert input_dimension % num_heads == 0, "Input dimensions must be a multiple of num_heads"

    self.input_dimension = input_dimension
    self.num_heads = num_heads
    self.single_head_dim = input_dimension // num_heads

    self.QMatrix = nn.Linear(input_dimension,input_dimension)       #Linear layer to hold the weight of Query Matrix  -> Projects input embeddings into query space
    self.KMatrix = nn.Linear(input_dimension,input_dimension)       #Linear layer to hold the weight of Key Matrix    -> Projects input embeddings into Key space
    self.VMatrix = nn.Linear(input_dimension,input_dimension)       #Linear layer to hold the weight of Value Matrix  -> Projects input embeddings into Value space
    self.OutMatrix = nn.Linear(input_dimension,input_dimension)     #Linear layer to hold the concatenated result of all the attention heads in original embedding size input_dimensions

  def create_attention_heads(self,input_embedding):
    """View method is called to reshape the input embedding matrix. After reshape new shape of tensor is (batch_size,seq_length,self.num_heads,self.single_head_dim)
    The transpose method is called after reshape to switch the dimensions at index 1 and 2 respectively."""
    batch_size, seq_length, d_model = input_embedding.size()
    #View Method breaks the input tensor into separate heads so that model can attend to different parts of the sequence simultaneously.
    #Transpose method is a necessary step to ensure that individual heads are alligned for matrix multiplication which helps to calculate attention scores.
    return input_embedding.view(batch_size,seq_length,self.num_heads,self.single_head_dim).transpose(1,2)

  def calculate_attention_scores(self,Q, K, V, mask = None):
    #Calculate the attention score belwo
    #Scale the attention score by the square root of the dimension of the keys (single_head_dim).
    #This helps to stabilize the gradients during training by preventing the scores from growing too large.
    attention_score = torch.matmul(Q,K.transpose(-2,-1)) / math.sqrt(self.single_head_dim)
    if mask is not None:
      attention_score = attention_score.masked_fill(mask == 0,-1e9)
    #Apply the softmax function to the attention scores along the last dimension (seq_length).
    #This converts the scores into probabilities that sum to 1 across the sequence length, allowing the model to attend to different positions in the input sequence.
    attention_prob = torch.softmax(attention_score,dim=-1)
    output = torch.matmul(attention_prob,V)
    return output

  def combine_all_heads(self,output):
    batch_size,number_of_heads,seq_length,output_embedding_dim = output.size()
    #Contiguous: Ensures the tensor memory is contiguous, which is necessary for the next view operation.
    #View: Reshape the tensor back to original shape as input
    return output.transpose(1,2).contiguous().view(batch_size,seq_length,self.input_dimension)

  def forward(self,Q,K,V,mask=None):
    Q = self.create_attention_heads(self.QMatrix(Q))
    K = self.create_attention_heads(self.KMatrix(K))
    V = self.create_attention_heads(self.VMatrix(V))

    attention_out = self.calculate_attention_scores(Q,K,V,mask)
    output = self.OutMatrix(self.combine_all_heads(attention_out))
    return output

In [ ]:
class FeedForward(nn.Module):
  def __init__(self,input_dimension,hidden_dimension):
    super(FeedForward,self).__init__()
    self.first_layer = nn.Linear(input_dimension,hidden_dimension)
    self.second_layer = nn.Linear(hidden_dimension,input_dimension)
    self.relu = nn.ReLU()

  def forward(self,input_tensor):
    return self.second_layer(self.relu(self.first_layer(input_tensor)))

In [ ]:
class Encoder_Unit(nn.Module):
  def __init__(self,input_dimension,num_of_heads,hidden_dimension,dropout_rate):
    super(Encoder_Unit,self).__init__()
    self.attention_block = MultiHead_Attention(input_dimension,num_of_heads)
    self.feed_forward_block = FeedForward(input_dimension,hidden_dimension)
    self.normblock_one = nn.LayerNorm(input_dimension)
    self.normblock_two = nn.LayerNorm(input_dimension)
    self.dropout_layer = nn.Dropout(dropout_rate)

  def forward(self,input_tensor,mask):
    #Shape of INput tensor: (Batch_Size,Length_of_input_sequence,input_dimension)
    attention_output = self.attention_block(input_tensor,input_tensor,input_tensor,mask)
    input_tensor = self.normblock_one(input_tensor + self.dropout_layer(attention_output))
    feed_forward_output = self.feed_forward_block(input_tensor)
    input_tensor = self.normblock_two(input_tensor + self.dropout_layer(feed_forward_output))
    return input_tensor

In [ ]:
class Decoder_Unit(nn.Module):
  def __init__(self,input_dimension,num_of_heads,hidden_dimension,dropout_rate):
    super(Decoder_Unit,self).__init__()
    self.masked_attention_block = MultiHead_Attention(input_dimension,num_of_heads)
    self.cross_attention_block = MultiHead_Attention(input_dimension,num_of_heads)
    self.feed_forward_block = FeedForward(input_dimension,hidden_dimension)
    self.normblock_one = nn.LayerNorm(input_dimension)
    self.normblock_two = nn.LayerNorm(input_dimension)
    self.normblock_three = nn.LayerNorm(input_dimension)
    self.dropout_layer = nn.Dropout(dropout_rate)

  def forward(self,input_tensor,encoder_output,src_mask,target_mask):
    attention_output = self.masked_attention_block(input_tensor,input_tensor,input_tensor,target_mask)
    input_tensor = self.normblock_one(input_tensor + self.dropout_layer(attention_output))
    attention_output = self.cross_attention_block(input_tensor,encoder_output,encoder_output,src_mask)
    input_tensor = self.normblock_two(input_tensor + self.dropout_layer(attention_output))
    output = self.feed_forward_block(input_tensor)
    input_tensor = self.normblock_three(input_tensor + self.dropout_layer(output))
    return input_tensor

In [ ]:
# size_of_verification_data = 0.1
# size_of_test_data = 0.2

# from sklearn.model_selection import train_test_split

# X = data['padded_sequences']
# y = data['label']

# X_model, X_verify, y_model, y_verify = train_test_split(X,y,test_size=size_of_verification_data,shuffle=True)
# X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=size_of_test_data, shuffle=True)

In [ ]:
# print(f"Info of X_verify: {X_verify.info()} \n\n")
# print(f"Info of X_train: {X_train.info()} \n\n")
# print(f"Info of X_test: {X_test.info()} \n\n")

In [ ]:
# from torch.utils.data import DataLoader
# BATCH_SIZE = 16


# train_dataloader = DataLoader(list(zip(X_train,y_train)), shuffle=True, batch_size=BATCH_SIZE)
# for X_batch, y_batch in train_dataloader:
#     # print(X_batch, y_batch)
#     break

# test_dataloader = DataLoader(list(zip(X_test,y_test)), shuffle=True, batch_size=BATCH_SIZE)
# for X_batch, y_batch in test_dataloader:
#     # print(X_batch, y_batch)
#     break

# verify_dataloader = DataLoader(list(zip(X_verify,y_verify)), shuffle=True, batch_size=BATCH_SIZE)
# for X_batch, y_batch in verify_dataloader:
#     # print(X_batch, y_batch)
#     break

In [ ]:
# print(f"Length of Train Dataloader is: {len(train_dataloader)}")
# print(f"Length of Test Dataloader is: {len(test_dataloader)}")
# print(f"Length of verify Dataloader is: {len(verify_dataloader)}")

In [ ]:
# train_feature_batch, train_labels_batch = next(iter(train_dataloader))
# len(train_feature_batch), len(train_labels_batch)

In [ ]:
# pos_encodings = pos_encoding(max_sent_len,d_model,10000)

In [ ]:
# weights = torch.tensor([0, 10, 3, 0], dtype=torch.float) # create a tensor of weights
# torch.multinomial(weights, 4,replacement=True)